In [1]:
import os
import time
import emcee
import numpy as np
import pandas as pd
from scipy.integrate import quad, odeint
from scipy.interpolate import interp1d
from multiprocessing import Pool

# We select Hubble flow SN and their covariance matrix

In [2]:
Pantheonplus_Data = pd.read_csv("../../Data/Pantheon+SH0ES.dat", sep='\s+')

Pantheonplus_Data = Pantheonplus_Data[["CID", "zHD", "zCMB", "zHEL", "m_b_corr", 
                                                                     "m_b_corr_err_DIAG", "CEPH_DIST",
                                                                     "IS_CALIBRATOR", "USED_IN_SH0ES_HF"]]

Pantheonplus_covariance = pd.read_csv("../../Data/Pantheon+SH0ES_STAT+SYS.cov",
                                                                  sep='\s+').values.reshape(1701, 1701)

HFSN = Pantheonplus_Data.query("zHD>0.0233 and zHD<0.15")

ndim = HFSN.shape[0]
SN_cov = np.zeros((ndim, ndim))
SN_cov_index = HFSN.index
SN_cov = np.array([Pantheonplus_covariance[SN_cov_index[i], SN_cov_index[j]]
                                for i in range(0, ndim) for j in range(0, ndim)
                                ]).reshape(ndim, ndim)

inverse_SN_cov = np.matrix(SN_cov).I

# We construct the complete CC data

In [3]:
# BC03 data

CC_BC03_data = '../../Data/HzTable_MM_BC03.dat'
CC_BC03_cov_mat = '../../Data/CC_cov_mat.txt'

z_BC03, Hz_BC03, errHz_BC03 = np.genfromtxt(CC_BC03_data,
                                                                             comments='#',
                                                                             usecols=(0, 1, 2),
                                                                             unpack=True,
                                                                             delimiter=',')

CC_BC03_cov_mat = np.genfromtxt(CC_BC03_cov_mat)

# The other data

z_CC_rest = np.array([0.1, 0.17, 0.27, 0.4, 0.48, 0.88, 0.9, 1.3, 1.43, 1.53, 
                                   1.75, 0.07, 0.12, 0.2, 0.28, 0.47, 0.75])

Hz_CC_rest = np.array([69, 83, 77, 95, 97, 90, 117, 168, 177, 140, 202, 
                                      69, 68.6, 72.9, 88.8, 89, 98.8])

errHz_CC_rest = np.array([12, 8, 14, 17, 62, 40, 23, 17, 18, 14, 40, 19.6, 
                                          26.2, 29.6, 36.6, 49.6, 33.6])

z_CC = np.concatenate((z_BC03, z_CC_rest))
Hz_CC = np.concatenate((Hz_BC03, Hz_CC_rest))

len_CC = len(z_CC)
len_BC03 = len(z_BC03)
total_cov = np.zeros((len_CC, len_CC), dtype='float64')
total_cov[:len_BC03, :len_BC03] = CC_BC03_cov_mat
total_cov[len_BC03:, len_BC03:] = np.diag(errHz_CC_rest**2)

inv_cov_CC = np.linalg.inv(total_cov)

# Here is the 2DBAO data

In [4]:
z_BAO = np.array([0.11, 0.235, 0.365, 0.45, 0.47, 0.49, 0.51, 0.53, 
                              0.55, 0.57, 0.59, 0.61, 0.63, 0.65, 2.225])

theta_BAO = np.array([19.8, 9.06, 6.33, 4.77, 5.02, 4.99, 4.81, 4.29, 
                                     4.25, 4.59, 4.39, 3.85, 3.90, 3.55, 1.82])

sigma_BAO = np.array([1.05, 0.23, 0.22, 0.17, 0.25, 0.21, 0.17, 0.30, 
                                      0.25, 0.36, 0.33, 0.31, 0.43, 0.16, 0.21])

# LambdaCDM Cosmology

In [5]:
class LambdaCDM(object):
    """
    This class gives the physical quantities, such as H/d_L/Modulus/chi2
    
    c : the light speed, unit km/s
    H0 : the present Hubble constant, unit km/s/Mpc
    Magnitude : the absolute magnitude
    H0MPC : H0 in unite of /Mpc
    """
    c = 2.998e5  #  km/s

    def __init__(self,
                 H0=None,
                 Magnitude=None,
                 rd=None,
                 Omegam=None,
                 Omegak=None):
        self.update_parameters(H0, Magnitude, rd, Omegam, Omegak)

    def update_parameters(self,
                          H0=None,
                          Magnitude=None,
                          rd=None,
                          Omegam=None,
                          Omegak=None):

        if (H0 is not None):
            self.H0 = H0
            self.H0Mpc = H0 / self.c

        if (Magnitude is not None):
            self.Magnitude = Magnitude

        if (rd is not None):
            self.rd = rd

        if (Omegam is not None):
            self.Omegam = Omegam

        if (Omegak is not None):
            self.Omegak = Omegak
            self.OmegaL = 1 - self.Omegam - self.Omegak

    def Hofz(self, z):

        Ez = np.sqrt(self.OmegaL + self.Omegam * (1 + z)**3 + self.Omegak *
                            (1 + z)**2)

        return Ez

    def Hofa(self, a):

        Ea = np.sqrt(self.OmegaL + self.Omegam / a**3 + self.Omegak / a**2)

        return Ea

    """
    calculate the Comoving_distance
    zcmb : ndarray or pandas Series
    """

    def Comoving_distance(self, zcmb):

        dchibydz = lambda z: 1 / self.Hofz(z)

        if ((type(zcmb) == np.ndarray)
                | (type(zcmb) == pd.core.series.Series)):

            I = np.array([quad(dchibydz, 0, nz)[0] for nz in zcmb])
            
        else:
            
            I = quad(dchibydz, 0, zcmb)[0]

        if (self.Omegak is not None):

            if (self.Omegak > 1e-20):
                I = np.sinh(I * self.Omegak**0.5) / self.Omegak**0.5

            elif (self.Omegak < -1e-20):
                I = np.sin(I * (-self.Omegak)**0.5) / (-self.Omegak)**0.5

            else:
                I = I
                
        res = I / self.H0Mpc  # here unit is  Mpc
        return res

    """
    calculate the distance Modulus
    zhd : ndarray or pandas Series
    zhel : ndarray or pandas Series
    """
    def distance_Modulus_model(self, zhd, zhel):
        
        def comovdistance_interpolation(zarray):
            zlist = np.logspace(-1.7, 0.4, 80)
            comovdistance = self.Comoving_distance(zlist)
            interp_func = interp1d(zlist, comovdistance, kind="cubic")
            return interp_func(zarray)

        Luminosity_distance = (1 + zhel) * comovdistance_interpolation(zhd)

        return 5 * np.log10(Luminosity_distance) + 25

    """
    calculate the angular BAO
    z : ndarray or pandas Series
    rd: the sound horizon unit: Mpc
    """
    def thetaBAO(self, z):
        return self.rd / self.Comoving_distance(z) * 180 / np.pi

    """
    calculate the chi square of SN
    """
    def chi2_SN(self):

        mu = self.distance_Modulus_model(HFSN["zHD"], HFSN["zHEL"])

        delta_matrix = np.matrix(HFSN["m_b_corr"] - mu - self.Magnitude)

        chi2 = delta_matrix * inverse_SN_cov * delta_matrix.T

        return chi2[0, 0]

    """
    calculate the chi square of BAO
    """
    def chi2_BAO(self):
        chi2 = sum(((self.thetaBAO(z_BAO) - theta_BAO) / sigma_BAO)**2)
        return chi2

    """
    calculate the chi square of CC
    """
    def chi2_CC(self):
        Hz = self.Hofz(z_CC) * self.H0
        delta_H = Hz - Hz_CC
        chi2 = np.dot(np.dot(delta_H, inv_cov_CC), delta_H)
        return chi2
    
    """
    the total chi square
    """
    def chi2_total(self):
        return self.chi2_SN() + self.chi2_BAO() + self.chi2_CC()

# PAge Cosmology

In [6]:
class PAge(LambdaCDM):
    c = 2.998e5  #km/s/Mpc

    def __init__(self,
                      H0=None,
                      Magnitude=None,
                      rd=None,
                      page=None,
                      eta=None,
                      Omegak=None):
        self.update_parameters(H0, Magnitude, rd, page, eta, Omegak)
        self.set_background()

    def update_parameters(self,
                                          H0=None,
                                          Magnitude=None,
                                          rd=None,
                                          page=None,
                                          eta=None,
                                          Omegak=None):

        if (H0 is not None):
            self.H0 = H0
            self.H0Mpc = H0 / self.c

        if (Magnitude is not None):
            self.Magnitude = Magnitude

        if (rd is not None):
            self.rd = rd

        if (page is not None):
            self.page = page

        if (eta is not None):
            self.eta = eta

        if (Omegak is not None):
            self.Omegak = Omegak

    def set_background(self):
        self.tlist = np.linspace(0.01, 1.3, 100)
        self.alist = self.aoft(self.tlist)
        self.zlist = 1 / self.alist - 1
        self.tofa = interp1d(self.alist, self.tlist, kind='cubic')  #calculate the t and a relation of PAge
        self.tofz = interp1d(self.zlist, self.tlist, kind='cubic')
        self.Hlist = self.Hoft(self.tlist)
        self.Hofz = interp1d(self.zlist, self.Hlist, kind='cubic')

    def Hoft(self, t):
        page, eta = self.page, self.eta
        Ht = 1 + (2. / 3.) * (1 - eta * t / page) * (1. / t - 1. / page)
        return Ht

    def aoft(self, t):
        page, eta = self.page, self.eta
        at = (t / page)**(2.0 / 3.0) * np.exp(eta / 3.0 * ((t / page)**2 - 1) +
                                                                (page - (2.0 / 3.0) *
                                                                (1.0 + eta)) * (t / page - 1.0))
        return at

    def chi(self, z):

        inverse_a = lambda t: 1 / self.aoft(t)

        t0 = self.page
        tz = self.tofa(1 / (1 + z))
        Integral = -quad(inverse_a, t0, tz)[0]

        return Integral

    def Comoving_distance(self, zcmb):

        if ((type(zcmb) == np.ndarray) |
            (type(zcmb) == pd.core.series.Series)):
            
            I = np.array([self.chi(nz) for nz in zcmb])
            
        else:
            I = self.chi(zcmb)

        if (self.Omegak > 1e-20):
            I = np.sinh(I * self.Omegak**0.5) / self.Omegak**0.5
            
        elif (self.Omegak < -1e-20):
            I = np.sin(I * (-self.Omegak)**0.5) / (-self.Omegak)**0.5
            
        else:
            I = I

        res = I / self.H0Mpc  # unit Mpc
        return res

# PDE Cosmology

In [7]:
class PDE(LambdaCDM):
    
    def __init__(self,
                       H0=None,
                       Magnitude=None,
                       rd=None,
                       Omegam=None,
                       Delta=None,
                       zc=None,
                       beta=None):
        self.update_parameters(H0, Magnitude, rd, Omegam, Delta, zc, beta)

    def update_parameters(self,
                                          H0=None,
                                          Magnitude=None,
                                          rd=None,
                                          Omegam=None,
                                          Delta=None,
                                          zc=None,
                                          beta=None):

        if (H0 is not None):
            self.H0 = H0
            self.H0Mpc = H0 / self.c

        if (Magnitude is not None):
            self.Magnitude = Magnitude

        if (rd is not None):
            self.rd = rd

        if (Omegam is not None):
            self.Omegam = Omegam
            self.OmegaL = 1 - Omegam

        if (Delta is not None):
            self.Delta = Delta

        if (zc is not None):
            self.zc = zc

        if (beta is not None):
            self.beta = beta

    def Hofz(self, z):

        Ez = np.sqrt(self.Omegam * (1 + z)**3 + self.OmegaL *
                            (1 + self.Delta * np.exp(-(z / self.zc)**self.beta)))

        return Ez

    def Hofa(self, a):

        Ea = np.sqrt(self.Omegam / a**3 + self.OmegaL *
                            (1 + self.Delta * np.exp(-(
                            (1 / a - 1) / self.zc)**self.beta)))

        return Ea

    def Comoving_distance(self, zcmb):

        dchibydz = lambda z: 1 / self.Hofz(z)

        if ((type(zcmb) == np.ndarray)
                | (type(zcmb) == pd.core.series.Series)):

            I = np.array([quad(dchibydz, 0, nz)[0] for nz in zcmb])
            
        else:
            I = quad(dchibydz, 0, zcmb)[0]

        res = I / self.H0Mpc  # here unit is  Mpc
        return res

# Define  LambdaCDM log-likelihood and MCMC function

In [8]:
def LambdaCDM_loglikelihood(theta):
    H0, Magnitude, rd, Omegam = theta

    top_limit = theta > np.array([90, -18, 200, 1])
    bottom_limit = theta < np.array([40, -21, 100, 0])
    if (sum(top_limit) + sum(bottom_limit) != 0):
        return -np.inf

    chi2 = LambdaCDM(H0, Magnitude, rd, Omegam, 0).chi2_total()

    return -0.5 * chi2


def LambdaCDM_MCMC_run(nsteps, path):
    os.environ["OMP_NUM_THREADS"] = "1"

    pool = Pool(5)

    nsteps = nsteps
    initial = np.random.random([12, 4]) * 0.02 + np.array([70, -19.2, 147, 0.3])
    nwalkers, ndim = initial.shape

    savefile = path
    backend = emcee.backends.HDFBackend(savefile)
    backend.reset(nwalkers, ndim)
    sampler = emcee.EnsembleSampler(nwalkers,
                                    ndim,
                                    LambdaCDM_loglikelihood,
                                    backend=backend,
                                    moves=emcee.moves.StretchMove())
    sampler.run_mcmc(initial, nsteps, progress=True)

# Define PAge log-Likelihood

In [9]:
def PAge_loglikehood(theta):
    H0, Magnitude, rd, page, eta = theta

    top_limit = theta > np.array([90, -18, 200, 1.2, 1])
    bottom_limit = theta < np.array([40, -21, 100, 0.7, -1])
    if (sum(top_limit) + sum(bottom_limit) != 0):
        return -np.inf

    chi2 = PAge(H0, Magnitude, rd, page, eta, 0).chi2_total()

    return -0.5 * chi2


def PAge_MCMC_run(nsteps, path):
    os.environ["OMP_NUM_THREADS"] = "1"

    nsteps = nsteps
    pool = Pool(5)
    initial = np.random.random([15, 5]) * 0.02 + np.array([70, -19.2, 147, 0.96, 0.3])
    nwalkers, ndim = initial.shape

    savefile = path
    backend = emcee.backends.HDFBackend(savefile)
    backend.reset(nwalkers, ndim)
    sampler = emcee.EnsembleSampler(nwalkers,
                                    ndim,
                                    PAge_loglikehood,
                                    backend=backend,
                                    moves=emcee.moves.StretchMove())
    sampler.run_mcmc(initial, nsteps, progress=True)

# Define PDE log-likelihood

In [10]:
def PDE_loglikehood(theta):
    H0, Magnitude, rd, Omegam, Delta, zc, beta = theta

    top_limit = theta > np.array([90, -18, 200, 1, 1, 1, 4])
    bottom_limit = theta < np.array([40, -21, 100, 0.0, -1, 0.0233, -4])
    if (sum(top_limit) + sum(bottom_limit) != 0): return -np.inf

    chi2 = PDE(H0, Magnitude, rd, Omegam, Delta, zc, beta).chi2_total()

    return -0.5 * chi2


def PDE_MCMC_run(nsteps, path):
    os.environ["OMP_NUM_THREADS"] = "1"

    nsteps = nsteps
    pool = Pool(5)

    initial = np.random.random([14, 7]) * 0.02 + np.array([70, -19.2, 147, 0.3, 0.3, 0.1, 2])
    nwalkers, ndim = initial.shape

    savefile = path
    backend = emcee.backends.HDFBackend(savefile)
    backend.reset(nwalkers, ndim)
    sampler = emcee.EnsembleSampler(nwalkers,
                                    ndim,
                                    PDE_loglikehood,
                                    backend=backend,
                                    moves=[
                                                  (emcee.moves.DEMove(), 0.8),
                                                  (emcee.moves.DESnookerMove(), 0.2)])
    sampler.run_mcmc(initial, nsteps, progress=True)

# Here we run MCMC 

In [11]:
Run_LambdaCDM = False
Run_PAge = False
Run_PDE = False

if __name__ == '__main__':
    
    if (Run_LambdaCDM == True):
        print("Now run LambdaCDM MCMC!")
        LambdaCDM_MCMC_run(12000,
                                                  '../../Chains/IDL=HFSN+2DBAO+CC/LambdaCDM.h5')

    if (Run_PAge == True):
        print("Now run PAge MCMC!")
        PAge_MCMC_run(15000, '../../Chains/IDL=HFSN+2DBAO+CC/PAge.h5')

    if (Run_PDE == True):
        print("Now run PDE MCMC!")
        PDE_MCMC_run(120000, '../../Chains/IDL=HFSN+2DBAO+CC/PDE.h5')